## scRNAseq: Epithelial-Mesenchymal Transition in Cancer
Dataset from: [Cook and Vanderhyden 2020](https://www.nature.com/articles/s41467-020-16066-2)

Re-aligned and de-multiplexed data, with unspliced RNA for RNAvelocity

In [ ]:
import os
currDir = os.getcwd()
settingsDir = currDir

In [ ]:
all_runs = ["A549-TGFB1",
            "DU145-TGFB1",
            "OVCA420-EGF", "OVCA420-TGFB1", "OVCA420-TNF"]

run_int = 0
run_name = all_runs[run_int]
dataset_name = "3-Cook"

In [ ]:
settingsDir

In [ ]:
file_name = run_name
run_name

## Run Settings

Import Cook functions

In [ ]:
cookFunctionDir = os.path.dirname(os.path.dirname(currDir))+"/3-Cook/Code/"
os.chdir(cookFunctionDir)
%run Cook_functions.ipynb

sc.settings.set_figure_params(dpi=150, figsize=[5,5])
plt.rcParams['figure.figsize']=(5,5)

# Directories
dataDir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(cookFunctionDir))))+"/RNAvelocity-datafiles/"
dataDir_changeTo = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(cookFunctionDir))))+"/EMT-in-cancer-datafiles/3-Cook-realigned/"
figDir_base = os.path.dirname(os.path.dirname(settingsDir))+"/3-Cook-realigned/Figures/"

settingsCsv = "Cook-realigned_run_settings.xlsx"

Settings for current run

In [ ]:
df = pd.read_excel(settingsDir+"/"+settingsCsv, engine='openpyxl')
run_settings = df.loc[df['run_name'] == run_name].squeeze()

total_counts_cutoff = run_settings["total_counts_cutoff"]
mito_cutoff = run_settings["mito_cutoff"]
ribo_cutoff = run_settings["ribo_cutoff"]
leiden_resolution = run_settings["leiden_resolution"]
EMT_clusters_in_order = run_settings["EMT_clusters_in_order"].split(",")
EMT_clusters_in_order = [int(i) for i in EMT_clusters_in_order]
pseudotime_DC = run_settings["pseudotime_DC"]
pseudotime_DC_rootNodes = run_settings["pseudotime_DC_rootNodes"]
run_E_markers = run_settings["run_E_markers"].split(", ") if str(run_settings["run_E_markers"]) != "nan" else ""
run_M_markers = run_settings["run_M_markers"].split(", ") if str(run_settings["run_M_markers"]) != "nan" else ""

## Part 1: IMPORT DATA, FILTERING, AND NORMALIZATION

In [ ]:
# Figure directories
figDir = figDir_base + "1-Filtering and Normalization/"+run_name+"/"
sc.settings.figdir = figDir

Import raw data; only keep forward timepoints

In [ ]:
adata = sc.read_h5ad(dataDir+"_h5ad/"+run_name+"/0-Demultiplexed.h5ad")

# Only keep forward timepoints
adata = adata[[x in ['#0_0d', '#1_8h', '#2_1d', '#3_3d', '#4_7d'] for x in adata.obs["Timepoint"]]].copy()
labelDict = {'#0_0d': '0) 0d',
             '#1_8h': '1) 8h',
             '#2_1d': '2) 1d',
             '#3_3d': '3) 3d',
             '#4_7d': '4) 7d'}
adata.obs["Timepoint"] = adata.obs["Timepoint"].replace(to_replace=labelDict)

In [ ]:
# Reset datadir to save in EMT-in-cancer-datafiles
dataDir = dataDir_changeTo

In [ ]:
# For OVCA420 cell lines specifically, they were contaminated with another cell line
# Remove the other cell line - cells already selected

if "OVCA420" in run_name:
    os.chdir(settingsDir)
    cells_to_keep = pd.read_csv("_withoutContaminantCellLine_"+run_name+".csv")["0"].to_list()
    cells_to_keep_bool = [True if currGene in cells_to_keep else False for currGene in adata.obs.index]
    adata = adata[cells_to_keep_bool].copy()

In [ ]:
adata

### Filtering

Note: lots of basic filtering has already been done \
during demultiplexing & categorizing step

In [ ]:
# ERCC, filter by cell number and gene count
adata = filterData(adata)

In [ ]:
# Gene metric plots
sc.pl.violin(adata, ['n_genes', 'total_counts', 'pct_counts_mito_gene'], jitter=0.4, multi_panel=True, save=" - n_genes, n_counts, perc_mito.png")
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mito_gene', save=" - pct_counts_mito_gene vs total_counts.png")
sc.pl.scatter(adata, x='total_counts', y='pct_counts_ribo_gene', save=" - pct_counts_ribo_gene vs n_counts.png")
sc.pl.scatter(adata, x='pct_counts_ribo_gene', y='pct_counts_mito_gene', save=" - pct_counts_mito_gene vs pct_counts_ribo_gene.png")

In [ ]:
adata.obs["Timepoint"].value_counts()

In [ ]:
saveFilteredData(adata, run_name)

### Normalization and HiVar

In [ ]:
# Total-count normalize to 10,000 counts per cell, natural log
sc.pp.normalize_total(adata, target_sum=1e4)
# Set raw
adata.raw = adata
adata.write(dataDir+"_h5ad/"+run_name+"/1-Filtered_and_Normalized_allGenes.h5ad")

# Log and scale data
sc.pp.log1p(adata)
adata.raw = adata
# Add cell cycle score
cellCycle_g1S_genes, cellCycle_g2M_genes = cell_cycle_genes(adata)
adata.X = adata.X.astype('<f8')
sc.tl.score_genes_cell_cycle(adata, s_genes=cellCycle_g1S_genes, g2m_genes=cellCycle_g2M_genes, use_raw=False)
adata.X = adata.X.astype('<f4')

# COOK UNIQUE
# Regress out batch effects with Combat
sc.pp.combat(adata, key="Mix")

# Highly variable genes (note: expects log data)
sc.pp.highly_variable_genes(adata)
adataHiVar = adata[:, adata.var.highly_variable].copy()

# Regress out effects of total counts per cell and percent mito AND cell cycle
sc.pp.regress_out(adataHiVar, ['total_counts','pct_counts_mito_gene', 'S_score', 'G2M_score'])
sc.pp.scale(adataHiVar, max_value=10)
adataHiVar.write(dataDir+"_h5ad/"+run_name+"/1-Filtered_and_Normalized_hiVarGenes.h5ad")

## Part 2: CLUSTERING

In [ ]:
# Figure directories
figDir = figDir_base + "/2-Clustering/"+run_name
sc.settings.figdir = figDir
dataSumDir = "/Users/meilumcd/Desktop/EMT-in-cancer/3-Cook-realigned/DataSummaries/2-Clustering/"

In [ ]:
# Normalized genes only (for this code, necessary to pass to 1 function later)
adata_normalized = sc.read_h5ad(dataDir+"_h5ad/"+run_name+"/1-Filtered_and_Normalized_allGenes.h5ad")
# pandas df with uppercase genes
df_normalized = adata_normalized.to_df()
df_normalized.columns = map(str.upper, df_normalized.columns)

In [ ]:
# PCA calculation
sc.tl.pca(adataHiVar, svd_solver='arpack')
sc.pl.pca(adataHiVar, color=["total_counts","n_genes","pct_counts_mito_gene","pct_counts_ribo_gene"], save=" - total_counts, n_genes, perc_mito, perc_ribo.png")
sc.pl.pca(adataHiVar, color=["Timepoint"], save=" - Timepoints.png")
# sc.pl.pca_variance_ratio(adataHiVar, log=True)

# UMAP calculation
sc.pp.neighbors(adataHiVar)
sc.tl.umap(adataHiVar)

In [ ]:
sc.pl.umap(adataHiVar, color=["total_counts","n_genes"], save=" - total_counts, n_genes.png")
sc.pl.umap(adataHiVar, color=["pct_counts_mito_gene","pct_counts_ribo_gene","phase"], save=" - perc_mito, perc_ribo, cell cycle.png")
sc.pl.umap(adataHiVar, color=["Mix"], save=" - batch.png")
sc.pl.umap(adataHiVar, color=["Timepoint"], palette="coolwarm_r", save=" - Timepoint.png")

In [ ]:
adataHiVar = leiden(adataHiVar, leiden_resolution, EMT_clusters_in_order, run_name)
adataHiVar = leiden_marker_genes(adataHiVar, dataset_name, run_name)
leiden_marker_genes_highlight(run_name, df_normalized)

In [ ]:
saveClusteredData(adataHiVar, run_name)

## Part 3: TRAJECTORY INFERENCE

In [ ]:
# figDir
figDir = figDir_base + "3-Trajectory Inference/"+run_name+"/"
sc.settings.figdir = figDir

Calculate pseudotime

In [ ]:
adataHiVar = diffmap(adataHiVar)
sc.pl.diffmap(adataHiVar, color=["leiden_label","TGFBI"], components=['2,3'])

In [ ]:
# Find best root nodes: highest values of diffmap DC

# THESE INDEXES ARE CURRENTLY BUGGED IN scanpy 1.9.1 (but not 1.8.2)
diffmap_dim_DC1 = np.asarray([cell_dim[1] for cell_dim in adataHiVar.obsm['X_diffmap']])
diffmap_dim_DC2 = np.asarray([cell_dim[2] for cell_dim in adataHiVar.obsm['X_diffmap']])

if run_name == "A549-TGFB1":
    # Want subset: DC2 low, DC1 low
    # All DC2 sorted, low values; remove DC1 high values
    DC1_low = diffmap_dim_DC1.argsort()[:1500]
    DC2_minus_DC1low = [i for i in diffmap_dim_DC2.argsort() if i not in DC1_low]
    root_nodes = DC2_minus_DC1low[:5]
    
else:
    root_nodes = pseudotime_rootNodes(adataHiVar, pseudotime_DC, pseudotime_DC_rootNodes, run_name, numRoots=50)

adataHiVar, df_pseudotime = pseudotime_mean(adataHiVar, root_nodes)
sc.pl.diffmap(adataHiVar, color=['dpt_pseudotime_mean'], components=['2,3'], save=" - Pseudotime Mean.png")

In [ ]:
pseudotime_per_cluster(adataHiVar, EMT_clusters_in_order, run_name, df_pseudotime)

In [ ]:
# Save h5ad
adata.write(dataDir+"_h5ad/"+run_name+"/3-Pseudotime.h5ad")

In [ ]:
# adata_beforeRNAvelo = adataHiVar.copy()

## Part 4: RNA VELOCITY

In [ ]:
import scvelo as scv
import shutil
scv.settings.figdir = figDir_base + "/4-RNA Velocity/"+run_name+"/"
figDir = figDir_base + "/4-RNA Velocity/"+run_name+"/"

In [ ]:
# Data overview: proportions of spliced to unspliced

scv.pl.proportions(adataHiVar, groupby="leiden_label", save="- spliced-to-unspliced.png")

In [ ]:
# Preprocessing
scv.pp.filter_and_normalize(adataHiVar) # Unnecessary?
scv.pp.moments(adataHiVar)

# Calculating velocity - there are different modes: stochastic, deterministic, dynamical
scv.tl.recover_dynamics(adataHiVar, n_jobs = 11) # Required for dynamical
scv.tl.velocity(adataHiVar, use_highly_variable=False, mode="dynamical")
    # actually not sure if highly_variable=False changes since adataHiVar is only for highly variable genes
scv.tl.velocity_graph(adataHiVar)
scv.tl.velocity_confidence(adataHiVar)
# Use dynamical, slower but produces more genes than other methods: https://github.com/theislab/scvelo/issues/158

In [ ]:
# Genes with unspliced counts in min 3 cells:

# first: each nonzero element, its row entry
    # A rows total
# second: each nonzero element, its column entry
    # B columns total
# unspliced matrix is A x B -- rows x columns

# cells are already filtered
# therefore, find genes with at least 3 cells of nonzero expression

# each number is a gene's index
# number listed >=3 times, that gene has unspliced counts
unspliced_indices = pd.Series(adataHiVar.layers["unspliced"].nonzero()[1])

print(np.sum(unspliced_indices.value_counts() >= 3))

unspliced_indices_min3cells = (unspliced_indices.value_counts() >= 3)
unspliced_indices_min3cells = unspliced_indices_min3cells[unspliced_indices_min3cells]
genes_with_unspliced_counts = [adataHiVar.var_names[i] for i in unspliced_indices_min3cells.index]

In [ ]:
# # Number of genes with velocity information
numGeneswVelocityInformation = np.sum(~np.isnan(adataHiVar.var['fit_likelihood']))
# numGeneswVelocityInformation

# Number of genes actually used in velocity model
numVelocityGenes = adataHiVar.var['velocity_genes'].sum()
numVelocityGenes

In [ ]:
# Sanity check:

# View top ~10 genes that contribute the most to the velocity model
genes_topLikelihood = adataHiVar.var['fit_likelihood'].sort_values(ascending=False).index[:20]
scv.pl.scatter(adataHiVar, genes_topLikelihood, color="leiden_label", ncols=3, linewidth=2, save="- likelihood, top genes.png")

# "Look at the phase portraits of genes with the highest likelihood. These contribute the most to the overall fit."
# https://github.com/theislab/scvelo/discussions/657

In [ ]:
# Velocity model overview
scv.pl.velocity_embedding_stream(adataHiVar, color=["leiden_label"], arrow_size=1.2, legend_loc="right margin", save="- umap - HQ velocity stream embedding.png")

In [ ]:
# RNAvelocity model overview
# Note: dpt_pseudotime was determined previously, unrelated to RNAvelocity
scv.pl.velocity_embedding_stream(adataHiVar, color=["leiden_label", "Timepoint", "dpt_pseudotime_mean"], save="- umap - velocity with clusters, timepoints, pseudotime.png")
scv.pl.scatter(adataHiVar, c=['leiden_label', 'Timepoint', 'velocity_length'], perc=[5, 95], save="- scatter - velocity vector length.png")
scv.pl.scatter(adataHiVar, c=['velocity_confidence'], perc=[5, 95])


### Differential RNAvelocity

In [ ]:
scv.tl.rank_velocity_genes(adataHiVar, groupby='leiden_label', n_genes=numGeneswVelocityInformation)

In [ ]:
# Differential velocity genes

scv.tl.rank_dynamical_genes(adataHiVar, groupby='leiden_label', n_genes=numGeneswVelocityInformation)
df_DiffVel_genes = pd.DataFrame(adataHiVar.uns['rank_dynamical_genes']['names'])
df_DiffVel_scores = pd.DataFrame.from_records(adataHiVar.uns['rank_dynamical_genes']['scores'])

DV_genes_overview = scv.get_df(adataHiVar, 'rank_dynamical_genes/names')
DV_genes_overview.head(10)

In [ ]:
# Differential velocity genes by cluster

diffVel_E_genes = pd.Series(data=df_DiffVel_scores["E"].values, index=df_DiffVel_genes["E"])
diffVel_I_genes = pd.Series(data=df_DiffVel_scores["I"].values, index=df_DiffVel_genes["I"])
diffVel_M_genes = pd.Series(data=df_DiffVel_scores["M"].values, index=df_DiffVel_genes["M"])

# Select top genes
diffVel_E_genes = diffVel_E_genes[diffVel_E_genes > .25]
diffVel_I_genes = diffVel_I_genes[diffVel_I_genes > .25]
diffVel_M_genes = diffVel_M_genes[diffVel_M_genes > .25]

In [ ]:
# Spearman's
scv.tl.rank_velocity_genes(adataHiVar, groupby='leiden_label', n_genes=numGeneswVelocityInformation)
velocityGenes_spearmans = adataHiVar.var[~np.isnan(adataHiVar.var['spearmans_score'])]['spearmans_score']

In [ ]:
# Only keep genes above Spearman's correlation cutoff
velocityGenes_spearmans_aboveCutoff = velocityGenes_spearmans[velocityGenes_spearmans > 0.5]

diffVel_E_genes = diffVel_E_genes.loc[set(diffVel_E_genes.index).intersection(set(velocityGenes_spearmans_aboveCutoff.index))]
diffVel_I_genes = diffVel_I_genes.loc[set(diffVel_I_genes.index).intersection(set(velocityGenes_spearmans_aboveCutoff.index))]
diffVel_M_genes = diffVel_M_genes.loc[set(diffVel_M_genes.index).intersection(set(velocityGenes_spearmans_aboveCutoff.index))]

diffVel_E_genes = diffVel_E_genes.sort_values(ascending = False)
diffVel_I_genes = diffVel_I_genes.sort_values(ascending = False)
diffVel_M_genes = diffVel_M_genes.sort_values(ascending = False)

<!-- # Testing ranges and outputs
19 elements -- 0, 19
20 elements -- 0, 20
21 elements -- 0, 20, 21

39 elements -- 0, 20, 39
40 elements -- 0, 20, 40
41 elements -- 0, 20, 40, 41 -->

In [ ]:
# Visually inspect differential velocity gene graphs - can exclude any with seemingly poor model fit
scv.settings.figdir = figDir_base + "4-RNA Velocity/" + run_name + "/DVel genes, all unfiltered"
shutil.rmtree(scv.settings.figdir)
os.mkdir(scv.settings.figdir)

gene_sets = [diffVel_E_genes, diffVel_I_genes, diffVel_M_genes]
cluster_names = ["E", "I", "M"] 

for currCluster, currSet in enumerate(gene_sets):
    # 20 genes fits per png
    currRange = np.arange(0, len(currSet), 20).tolist()
    currRange.append(len(currSet))

    for i in range(0, len(currRange)-1):
        scv.pl.scatter(adataHiVar, list(currSet.index[currRange[i] : currRange[i+1]]), color="leiden_label", ncols=3, linewidth=2, show=False, save=" - "+cluster_names[currCluster]+" DiffVel Genes "+str(i*20)+"-"+str((i+1)*20)+", unfiltered.png")

### Export DiffVel genes

In [ ]:
scv.settings.figdir = figDir_base + "4-RNA Velocity/" + run_name

# Export list of DV genes
os.chdir(dataSumDir)
diffVel_E_genes.to_csv("Cluster DiffVel Genes - "+run_name+", E.csv", header=["cluster fit_likelihood"])
diffVel_I_genes.to_csv("Cluster DiffVel Genes - "+run_name+", I.csv", header=["cluster fit_likelihood"])
diffVel_M_genes.to_csv("Cluster DiffVel Genes - "+run_name+", M.csv", header=["cluster fit_likelihood"])

In [ ]:
# https://github.com/scverse/anndata/issues/628
# for some reason there is an error when saving the h5ad as-is; this person's solution works
adataHiVar.obs.to_csv("temporary.csv")
metadata = pd.read_csv("temporary.csv")
os.remove("temporary.csv")
metadata.set_index("Unnamed: 0", inplace=True)
adataHiVar.obs = metadata

# Save h5ad w rnavelocity
adataHiVar.write(dataDir+"_h5ad/"+run_name+"/4-RNA_velocity.h5ad")